In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import uniform
import os
from tqdm import tqdm

In [3]:
INPUT_FILE = '../data/raw/articles_links.csv'
OUTPUT_FILE = '../data/raw/raw_dataset.csv'
OUTPUT_COLUMNS = ['category', 'disease', 'link', 'data']
SAVE_INTERVAL = 25
WAIT_RANGE = (1, 3)
USER_AGENT = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 Chrome/119.0 Safari/537.36'

In [4]:
def crawl_text(url):
    try:
        headers = {'User-Agent': USER_AGENT}
        response = requests.get(url, headers=headers, timeout=15)
        response.encoding = 'utf-8'
        if response.status_code != 200:
            return f"❌ Lỗi HTTP {response.status_code}"

        soup = BeautifulSoup(response.text, 'html.parser')

        for tag in soup(['script', 'style', 'nav', 'footer', 'header', 'aside']):
            tag.decompose()

        content_div = soup.find(id='symptom')
        if not content_div:
            return "⚠️ Không tìm thấy phần id='symptom'"

        text = content_div.get_text(separator='\n', strip=True)
        text = ' '.join(text.split())
        return text

    except Exception as e:
        return f"❌ Lỗi: {e}"

In [5]:
print("📂 Đang đọc file đầu vào...")
try:
    df = pd.read_csv(INPUT_FILE)
    if not {'category', 'disease', 'link'}.issubset(df.columns):
        print(f"❌ Lỗi: File input phải có 3 cột 'category', 'disease', và 'link'")
        raise SystemExit

    print(f"✅ Đọc {len(df)} dòng dữ liệu. Các cột: {', '.join(df.columns)}")
except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file {INPUT_FILE}. Vui lòng upload file.")
    raise SystemExit

📂 Đang đọc file đầu vào...
✅ Đọc 1699 dòng dữ liệu. Các cột: category, disease, link


In [ ]:
results = []
crawled_links = set()
new_items_count = 0

if os.path.exists(OUTPUT_FILE):
    print(f"🔁 Phát hiện file cũ {OUTPUT_FILE}. Đang đọc để tiếp tục...")
    old_df = pd.read_csv(OUTPUT_FILE)
    if 'link' in old_df.columns:
        crawled_links = set(old_df['link'])
        print(f"✅ Đã có {len(crawled_links)} bài. Sẽ bỏ qua các link này.")
        results = old_df.values.tolist()
    else:
        print("⚠️ File output cũ không có cột 'link'. Sẽ crawl lại từ đầu.")
else:
    print("🌅 Bắt đầu crawl mới.")


print("🚀 Bắt đầu crawl...")
for _, row in tqdm(df.iterrows(), total=len(df), desc="Đang crawl"):
    category, disease, link = row['category'], row['disease'], row['link']

    if link in crawled_links:
        continue

    text = crawl_text(link)

    results.append([category, disease, link, text])
    crawled_links.add(link)
    new_items_count += 1

    if new_items_count > 0 and new_items_count % SAVE_INTERVAL == 0:
        temp_df = pd.DataFrame(results, columns=OUTPUT_COLUMNS)
        temp_df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
        tqdm.write(f"\n💾 Đã lưu tạm {len(results)} bài (thêm {new_items_count} bài mới) vào {OUTPUT_FILE}")

    sleep(uniform(*WAIT_RANGE))

In [6]:
pd.DataFrame(results, columns=OUTPUT_COLUMNS).to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
print(f"\n✅ Hoàn tất! Tổng cộng {len(results)} bài đã được lưu tại: {OUTPUT_FILE}")
print(f"Trong phiên chạy này đã crawl thêm được {new_items_count} bài mới.")


✅ Hoàn tất! Tổng cộng 1543 bài đã được lưu tại: /content/drive/MyDrive/nckh/raw_dataset.csv
Trong phiên chạy này đã crawl thêm được 0 bài mới.
